# @pytest.fixture와 @pytest.fixture()의 차이

> "파이썬 데코레이터"

- toc: true
- branch: master
- badges: true
- comments: true
- hide: true
- author: 한재수
- categories: [Python]

https://github.com/pytest-dev/pytest/blob/8.3.2/src/_pytest/fixtures.py#L1242

```python
@pytest.fixture
def f():
    # do something
    

# 위의 코드는 다음과 동일함
f = pytest.fixture(f)
```

`-` 반면에

```python
@pytest.fixture()
def f():
    # do something

    
# 위의 코드는 다음과 동일함
pyfixture = pytest.fixture()
f = pyfixture(f)
```

`-` 내 궁금증은 다음과 같음

`-` 어 근데 실제로 코딩할 땐 arguments 없는 상황 가정한다면

`-` 데코레이터로 @pytest.fixture라고 하든

`-` @pytest.fixture()라고 하든 동일한 기능을 수행했음

`-` 이거 어떻게 가능한 거임?????

`-` pytest에서 fixture 함수(=데코레이터 함수)는 기본적으로 2가지 형태로 오버로딩 되어있음

`-` 이를 설명하기 앞서 기본적인 데코레이터를 살펴보자

```python
def deco(func):
    # do something
```

`-` 데코레이터 함수의 첫 번째 인자는 당연히 데코레이터에 넣을 함수임. ㅇㅈ? 이걸 알면 끝났음

`-` 위에서 fixture 함수는 오버로딩이 2가지 형태로 되어있다 했음

`-` 첫 번째 경우를 보자

```python
@overload
def fixture(
    fixture_function: FixtureFunction,
    *,
    scope: _ScopeName | Callable[[str, Config], _ScopeName] = ...,
    params: Iterable[object] | None = ...,
    autouse: bool = ...,
    ids: Sequence[object | None] | Callable[[Any], object | None] | None = ...,
    name: str | None = ...,
) -> FixtureFunction: ...
```

`-` 첫 번째 인자인 fixture_function은 데코레이터에 넣을 함수를 뜻함 (그냥 func이라 해도 되는데 풀네임으로 적었다 생각하면 됨)

`-` 그리고 type은 FixtureFunction이다 ---> 이거 그냥  Callable TypeVar임.

`-` 그리고 리턴 type은 FixtureFunction이다 ---> 마찬가지로 그냥 Callable 오브젝트임 ㅇㅇ

`-` 그럼 이제 두 번째 형태를 보자

```python
@overload
def fixture(
    fixture_function: None = ...,
    *,
    scope: _ScopeName | Callable[[str, Config], _ScopeName] = ...,
    params: Iterable[object] | None = ...,
    autouse: bool = ...,
    ids: Sequence[object | None] | Callable[[Any], object | None] | None = ...,
    name: str | None = None,
) -> FixtureFunctionMarker: ...
```

`-` 첫 번째 꼴과 다름 점은 2가지가 존재함

`-` 1) fixture_function의 타입이 None이다

`-` 2) 리턴 타입이 FixtureFunctionMarker이다

`-` 참고로 FixtureFunctionMarker는 클래스임

`-` 자 이제 함수 내부를 살펴보자 (매우 간단함)

```python
def fixture(
    fixture_function=None,
    *,
    scope="function",
    params=None,
    autouse=False,
    ids=None,
    name=None,
):
    fixture_marker = FixtureFunctionMarker(
        scope=scope,
        params=tuple(params) if params is not None else None,
        autouse=autouse,
        ids=None if ids is None else ids if callable(ids) else tuple(ids),
        name=name,
        _ispytest=True,
    )
    # Direct decoration
    if fixture_function:
        return fixture_marker(fixture_function)
    return fixture_marker
```

`-` 주석은 생략했음, 타입도 생략

`-` 일단 fixture_marker를 생성하고 시작함

`-` 그리고 입력으로 fixture_function이 들어온다면 (None이 아니라면) fixture_marker(fixture_function)을 리턴함

`-` fixture_function이 None이라면 fixture_marker를 리턴함

`-` @pytest.fixture인 경우 f = pytest.fixture(f)가 된다

`-` 그럼 fixture_marker를 만듦

`-` if문에서 fixture_function이 None이 아니고 f임

`-` 그래서 아까 만들어둔 fixture_marker(f)를 리턴함

`-` 여기서 궁금한건 fixture_marker는 뭐냐는 거임

`-` 일단 fixture_marker는 FixtureFunctionMarker 클래스의 인스턴스다

`-` 이제 궁금한건 이 클래스의 __call__ 메서드가 반환하는게 무엇이냐임

`-` __call__ 메서드는 FixtureFunction 타입을 반홤함 ---> 아까 말했듯이 이거 그냥 함수임

`-` 즉, @pytest.fixture인 경우 그냥 일반적인 데코레이터와 다를바 없다

`-` 진짜 궁금한 건 @pytest.fixture()임

`-` 이건 코드상으로 다음과 같음

```python
pyfixture = pytest.fixture()
f = pyfixture(f)
```

`-` 여기서의 f와 pytest.fixture(f)가 동일함 (내 궁금증 설명하는 부분에서 말했다)

`-` 어떻게 동일할 수가 있지?????

`-` pyfixture = pytest.fixture()를 잘보자

`-` pytest.fixture()는 사실 pytest.fixture(fixture_function=None)과 같음 (함수의 default값을 생각하면 당연함)

`-` 함수 내부를 보면 `if fixture_function` 코드가 있음. 근데 None임

`-` 따라서 해당 if문은 스킵됨. (if fixture_function이 아니라 if fixture_function is not None이 더 좋을 것 같은데 이렇게 안한 이유가 있겠지?)

`-` 그래서 그냥 fixture_marker만 리턴함. 데코레이터가 @pytest.fixture일땐 fixture_marker(f)였음.

`-` 즉 pytest.fixture()(f) = fixture_marker(f)이고 pytest.fixture(f)도 fixture_marker(f)이다

`-` pytest.fixture()로 입력안하고 pytest.fixture로 입력하니 자체적으로 pytest.fixture()과 동일한 기능을 하게 만들어줌

`-` 따라서 @pytest.fixture()나 @pytest.fixture나 동일한 기능을 수행함